# LLM Bench Chart Development
This notebook lets us experiment with chart colors and layout for the LLM benchmark results.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
results = Path('llm-bench/llama-2-7b.Q4_0/results.jsonl')
df = pd.read_json(results, orient='records', lines=True)
df.head()

In [ ]:
BACKEND_COLORS = {
    'hip': 'tab:blue',
    'rocwmma': 'tab:orange',
    'vulkan': 'tab:green',
}

def _format_b_flag(b: str) -> str:
    if b.startswith('-b '):
        return 'b=' + b.split()[1]
    return b

def _make_label(build: str, fa: str, b: str, hiplt: str) -> str:
    backend = build.split('-', 1)[-1].upper()
    parts = []
    if hiplt:
        parts.append('hipBLASLt')
    if fa:
        parts.append('fa=1')
    if b:
        parts.append(_format_b_flag(b))
    return f'{backend} {' '.join(parts)}'.strip()

def plot_mode(df, mode):
    subset = df[df['mode'] == mode]
    fig, ax = plt.subplots(figsize=(8,5))
    for (build, fa, b, hiplt), grp in subset.groupby(['build','fa','b','hipblaslt']):
        backend = build.split('-', 1)[-1]
        color = BACKEND_COLORS.get(backend, 'tab:gray')
        label = _make_label(build, fa, b, hiplt)
        ax.plot(grp['value'], grp['tokens_per_sec'], marker='o', label=label, color=color)
    ax.set_title(f'Tokens/s – {mode.upper()} sweep')
    ax.set_xlabel('Tokens')
    ax.set_ylabel('Tokens/s')
    ax.grid(True, alpha=0.3, linestyle=':')
    ax.legend(bbox_to_anchor=(1.02,1), loc='upper left')
    plt.show()


In [ ]:
plot_mode(df, 'pp')

In [ ]:
plot_mode(df, 'tg')